# 项目概述

## 项目简介

    本项目使用网络爬虫工具，爬取拉勾网上Title为‘数据分析’的岗位信息，并利用Python对这些数据进行分析、统计和图形展现。同过统计计算和图示化等方式，解答数据分析这一岗位的一些问题，供已经工作和打算从事数据分析工作的童鞋作为参考。
    
## 数据介绍
    项目数据全部来自网络招聘平台拉勾网。之所以选择拉勾网作为本项目的数据源，主要是因为拉钩网上的岗位信息非常完整、整洁，极少存在信息的缺漏。并且几乎所有展现出来的信息都是非常规范化的，极大的减少了前期数据清理和数据整理的工作量。
    在本项目中，主要爬取了数据分析岗位的以下信息：

| 内容    | 字段        |
|:---------- |:------     |
|岗位名称 |title       |
|月薪     |month_salary|
|公司名称  |company     |
|所属行业  |industry    |
|公司规模  |scale       |
|融资阶段  |phase       |
|投资人    |investors   |
|所在城市  |city        |
|经验要求  |experience  |
|学历要求  |qualification|
|全职/兼职 |full_or_parttime|
|职位描述及任职要求|description|

## 目的
    本项目的主要目的是通过实际的数据来解答针对数据分析岗位的一些疑惑，给从事这一职业或者打算从事这一职业的童鞋一些参考性的意见。具体来说，主要回答了以下几个问题：

1. 数据分析师岗位需求的地域性分布；
2. 整个群体中薪酬分布的情况；
3. 不同城市数据分析师的薪酬情况是怎样的；
4. 该岗位对于工作经验的要求是怎样的；
5. 根据工作经验的不同，薪酬是怎样变化的；
6. 从用人单位的角度看，数据分析师应当具备哪些技能？
7. 掌握不同技能是否会对薪酬有影响？影响是怎样的？
    
## 技术和工具
    本项目主要分为两大部分，第一部分是数据爬取，采用的是集搜客网络爬虫工具。第二部分是数据分析，以python编程语言为基础。
    集搜客是一款用C++语言编写的网络爬虫软件，性能稳定，功能强大，操作简洁。通过鼠标点选和简单的命令即可完成爬虫的定制和运行，还可以将爬取到的数据存储为各种常见格式，非常便于后期的分析工作。
    数据分析部分主要使用pandas作为数据整理和统计分析的工具，matplotlib用于图形的可视化，seaborn库包用于图形美化。在进行技能需求分析的时候，使用了jieba作为分词工具包，并使用wordcloud包制作词云。

# 数据整理

In [2]:
    #加载相关库包，设置matplotlib的字体，使之能够支持中文
    import numpy as np
    import pandas as pd
    import seaborn as sns
    import matplotlib as mpl
    from matplotlib import pyplot as plt
    from matplotlib.font_manager import FontProperties
    %matplotlib inline
    
    #由于matplotlib默认字体不是中文字体，在进行中文显示的时候只有一个框框，因此必须显式的指定字体文件
    font_small = mpl.font_manager.FontProperties(fname=
                    "/Users/carrey/anaconda/lib/python2.7/site-packages/matplotlib/mpl-data/fonts/ttf/Vera.ttc",
                    size = 12)
    font_big = mpl.font_manager.FontProperties(fname=
                    "/Users/carrey/anaconda/lib/python2.7/site-packages/matplotlib/mpl-data/fonts/ttf/Vera.ttc",
                    size = 24)

In [3]:
    data = pd.read_excel('Data_Lagou_utf8.xlsx')
    print data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589 entries, 0 to 588
Data columns (total 21 columns):
_clueid             589 non-null int64
_fullpath           589 non-null object
_realpath           589 non-null object
_theme              589 non-null object
_middle             589 non-null object
_createdate         589 non-null object
_pageno             589 non-null int64
_actionno           0 non-null float64
_actionvalue        0 non-null float64
title               589 non-null object
month_salary        589 non-null object
company             588 non-null object
industry            588 non-null object
scale               588 non-null object
phase               588 non-null object
investors           117 non-null object
city                589 non-null object
experience          589 non-null object
qualification       589 non-null object
full_or_parttime    589 non-null object
description         589 non-null object
dtypes: float64(2), int64(2), object(17)
memory usage: 96.7

    数据集中总共有589条记录，21个变量。但是以下划线开头的这几个变量是爬虫在进行数据爬取时自动生成的记录字段，并无实际意义，应该清理掉。另外，考虑到拉勾网允许企业重复投放岗位信息，可能有部分岗位信息是重复的，也应该清理掉。

In [4]:
    #清理不需要的字段
    new_data = data.drop([u'_clueid', u'_fullpath', u'_realpath', u'_theme',
                            u'_middle',u'_pageno', u'_actionno', u'_actionvalue'],
                            axis = 1)
    
    #清理重复行
    clean_data = new_data.drop_duplicates(['company','title','description'])
    print clean_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 575 entries, 0 to 588
Data columns (total 13 columns):
_createdate         575 non-null object
title               575 non-null object
month_salary        575 non-null object
company             574 non-null object
industry            574 non-null object
scale               574 non-null object
phase               574 non-null object
investors           115 non-null object
city                575 non-null object
experience          575 non-null object
qualification       575 non-null object
full_or_parttime    575 non-null object
description         575 non-null object
dtypes: object(13)
memory usage: 62.9+ KB
None


    清除掉不需要的字段和重复行之后，数据集还有13个变量，575条记录。

In [5]:
    print clean_data.head()

           _createdate      title month_salary  company    industry  \
0  2016-09-06 19:39:34   营销数据分析专员        4k-5k      泡单词    教育,移动互联网   
1  2016-09-06 19:39:38       数据分析        3k-4k      赛百威  移动互联网,电子商务   
2  2016-09-06 19:39:43    数据分析工程师      10k-20k     多益网络       移动互联网   
3  2016-09-06 19:39:48     商业数据分析        4k-6k      欧舜港   O2O,移动互联网   
4  2016-09-06 19:39:54  数据分析&数据挖掘      10k-20k  OpenCom   移动互联网,O2O   

       scale  phase       investors city experience qualification  \
0    50-150人    天使轮             NaN   广州       经验不限         大专及以上   
1    50-150人  不需要融资             NaN   广州       经验不限         大专及以上   
2  500-2000人  不需要融资             NaN   广州     经验1-3年         本科及以上   
3      少于15人    未融资             NaN   广州     经验1-3年         大专及以上   
4     15-50人     A轮  文投创工场，滨海基金(A轮)   广州       经验不限         本科及以上   

  full_or_parttime                                        description  
0               全职  职位描述\n        岗位职责：1. 负责数据采集，进行统计分析处理（用户来源、活跃性...  
1             

    数据集中‘month_salary’字段的值是区间值，并且是字符类型。为了便于后面的量化分析，我们需要将该字段进行整理，用区间中值来代表薪酬情况。

In [6]:
    #定义一个处理函数，将'month_salary'的区间值转化为具体的数字
    def avg_salary(salary):
        try:
            s_list = salary.split('-')
            s_min = int(s_list[0][:-1])
            s_max = int(s_list[1][:-1])
            s_avg = float(s_min + s_max)/2
        except UnicodeEncodeError:
            s_list = salary.split('k')
            s_avg = float(int(s_list[0][:-1]))
        return s_avg
    
    #生成计算字段‘avg_month_salary’以存储计算出的月薪
    clean_data['avg_month_salary'] = clean_data['month_salary'].apply(avg_salary)

/Users/carrey/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


    字段‘experience’记录了每个岗位对工作经验的要求，总是以‘经验’两字开头，可以稍作整理。

In [7]:
    #干掉‘经验’二字
    def remove_first_2_words(word):
        return word[2:]
    
    clean_data['short_exp'] = clean_data['experience'].apply(remove_first_2_words)
    print clean_data.head()

/Users/carrey/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
